#Data Preparation

Install dan import package

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Read Dataset

In [2]:
data_restaurant="https://docs.google.com/spreadsheets/d/e/2PACX-1vTyCBQxA1HyRnQWj-RXf-_5DXKn3L90Td0aO5r8DsTtE7pO3IQbPVvcuhfaCmJvTwiD_Hhvw34Xzbya/pub?gid=2129966204&single=true&output=csv"
data_makanminum="https://docs.google.com/spreadsheets/d/e/2PACX-1vTyCBQxA1HyRnQWj-RXf-_5DXKn3L90Td0aO5r8DsTtE7pO3IQbPVvcuhfaCmJvTwiD_Hhvw34Xzbya/pub?gid=477290477&single=true&output=csv"

# membaca kedua sheet ke dalam DataFrame
data_resto = pd.read_csv(data_restaurant)
data_makmin = pd.read_csv(data_makanminum)

print(data_resto.head())
print(data_makmin.head())

  resto_id                                      nama_restoran  \
0   RT01J1                Mie Bandung Chinese Food, Ngampilan   
1   RT02J1  Soto Tauco Pekalongan Bu Iswi, Jalan Bimokurdo...   
2   RT03J1                  Bakso Urat dan Mie Ayam "Pak Mir"   
3   RT04J1  Soto Ayam Kampung Dan Soto Daging Sapi Pak Gendut   
4   RT05J1                                          Toba Tabo   

  preferensi_makanan lokasi_restoran  Harga_Rata-Rata_Makanan_di_Toko (Rp)  \
0            Chinese         3.23 km                                 39000   
1          Indonesia         3.01 km                                 19000   
2          Indonesia         1.44 km                                 18000   
3          Indonesia         0.83 km                                 22000   
4            Chinese         2.13 km                                 40000   

   rating_toko jenis_suasana toko_sering_diskon_(Ya/Tidak)  \
0          4.7        Santai                         Tidak   
1          3.7  

Memilih kolom yang digunakan

In [3]:
data_makmin.columns = data_makmin.columns.str.strip()

columns_to_select = ['makanan/minuman', 'kategori', 'deskripsi_rasa']
makanan_data = data_makmin[columns_to_select]

print("\nDataset dengan kolom terpilih:")
print(makanan_data.head())


Dataset dengan kolom terpilih:
  makanan/minuman       kategori                               deskripsi_rasa
0    Sate Klathak  Makanan Berat                   Gurih, pedas, juicy, smoky
1    Soto Kambing  Makanan Berat  Gurih, pedas, rempah, segar, lezat, berkuah
2    Sate Kambing  Makanan Berat      Gurih, pedas, juicy, smoky, asin, manis
3     Wedang Uwuh        Minuman                 Manis, hangat, rempah, segar
4          Bakpia        Jajanan                 Manis, lembut, gurih, kacang


In [4]:
#memilih kolom sheet data restoran
data_resto.columns = data_resto.columns.str.strip()

columns_select = ['nama_restoran', 'rating_toko', 'variasi_makanan']
restoran_data = data_resto[columns_select]

print("\nDataset dengan kolom terpilih:")
print(restoran_data.head())


Dataset dengan kolom terpilih:
                                       nama_restoran  rating_toko  \
0                Mie Bandung Chinese Food, Ngampilan          4.7   
1  Soto Tauco Pekalongan Bu Iswi, Jalan Bimokurdo...          3.7   
2                  Bakso Urat dan Mie Ayam "Pak Mir"          3.6   
3  Soto Ayam Kampung Dan Soto Daging Sapi Pak Gendut          3.8   
4                                          Toba Tabo          4.7   

                                     variasi_makanan  
0  Bakmi, Nasi Goreng, Mie Ayam, Kwetiaw, Bakso P...  
1  Bakso, Soto Tauco, Lontong, Es Teh, Es Jeruk, ...  
2         Bakso, Sop Ayam, Jajanan, Es Jeruk, Es Teh  
3  Soto Daging, Soto Ayam, Perkedel, Es Teh, Es J...  
4  Sate Babi, Babi Kecap, Bakmi Babi, Kopi, Es Ko...  


Pre processing

In [5]:
# Menangani nilai NaN dalam kolom 'deskripsi_rasa'
makanan_data['deskripsi_rasa'].fillna('', inplace=True)

# Encoding kategori makanan
label_encoder = LabelEncoder()
makanan_data['kategori_Encoded'] = label_encoder.fit_transform(makanan_data['kategori'])

# Membagi dataset menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(makanan_data['deskripsi_rasa'],
                                                    makanan_data['kategori_Encoded'],
                                                    test_size=0.2, random_state=42)

# Membuat vektor fitur dari deskripsi rasa menggunakan TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train).toarray()  # Konversi ke bentuk array
X_test_vec = vectorizer.transform(X_test).toarray() # Konversi ke bentuk array

<ipython-input-5-467fe049e757>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  makanan_data['deskripsi_rasa'].fillna('', inplace=True)
<ipython-input-5-467fe049e757>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  makanan_data['kategori_Encoded'] = label_encoder.fit_transform(makanan_data['kategori'])


# Membangun model rekomendasi makanan bedasarkan deskripsi rasa

In [6]:
# Membangun model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(X_train_vec.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
])

In [7]:
# Create and compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
# Train the model
model.fit(X_train_vec,
          y_train, epochs=100, validation_split=0.2)

Epoch 1/100
7/7 [==============================] - 3s 110ms/step - loss: 1.9327 - accuracy: 0.1869 - val_loss: 1.8867 - val_accuracy: 0.4400
Epoch 2/100
7/7 [==============================] - 0s 23ms/step - loss: 1.8261 - accuracy: 0.4091 - val_loss: 1.7901 - val_accuracy: 0.4400
Epoch 3/100
7/7 [==============================] - 0s 22ms/step - loss: 1.6947 - accuracy: 0.4949 - val_loss: 1.6595 - val_accuracy: 0.4400
Epoch 4/100
7/7 [==============================] - 0s 14ms/step - loss: 1.5606 - accuracy: 0.4697 - val_loss: 1.5146 - val_accuracy: 0.4400
Epoch 5/100
7/7 [==============================] - 0s 16ms/step - loss: 1.4296 - accuracy: 0.5000 - val_loss: 1.4166 - val_accuracy: 0.4400
Epoch 6/100
7/7 [==============================] - 0s 15ms/step - loss: 1.3300 - accuracy: 0.5051 - val_loss: 1.3604 - val_accuracy: 0.4400
Epoch 7/100
7/7 [==============================] - 0s 26ms/step - loss: 1.3149 - accuracy: 0.5051 - val_loss: 1.3082 - val_accuracy: 0.4400
Epoch 8/100
7/7 [==

In [9]:
#Evaluasi akurasi
test_loss, test_acc = model.evaluate(X_test_vec, y_test)
print('Test accuracy:', test_acc)

2/2 [==============================] - 0s 8ms/step - loss: 1.0766 - accuracy: 0.8065
Test accuracy: 0.8064516186714172


In [10]:
#Fungsi untuk merekomendasikan makanan/minuman bedasarkan inputan
def recommend_makanan(category, taste):
    # Vektorisasi input menggunakan TfidfVectorizer
    input_vec = vectorizer.transform([taste]).toarray()

    # Prediksi kategori
    category_pred = model.predict(input_vec)

    # Mendapatkan kategori yang diprediksi
    predicted_category = label_encoder.inverse_transform([category_pred.argmax()])[0]

    # Filter data berdasarkan kategori yang diprediksi
    category_data = makanan_data[makanan_data['kategori'] == category]

    # Vektorisasi deskripsi rasa dalam data latih
    X_train_category = vectorizer.transform(category_data['deskripsi_rasa']).toarray()

    # Hitung similarity scores antara input dan deskripsi makanan dalam data latih
    similarity_scores = cosine_similarity(input_vec, X_train_category)

    # Dapatkan indeks top 5 makanan yang paling mirip
    top_indices = similarity_scores.argsort()[0][-5:][::-1]

    # Dapatkan rekomendasi makanan berdasarkan kategori yang diprediksi
    recommended_food = category_data.iloc[top_indices]['makanan/minuman'].values

    return recommended_food

Inputan Dropdown button & free text (for testing)

- Dropdown button untuk inputan kategori
- Free text untuk inputan deskripsi rasa


In [11]:
# Contoh penggunaan
category = 'Jajanan'
taste = 'asin, gurih, renyah'
recommended_food = recommend_makanan(category, taste)

print("Rekomendasi makanan bedasarkan kategori '{}' dengan rasa '{}':".format(category, taste))
for idx, food in enumerate(recommended_food, 1):
    print("{}. {}".format(idx, food))

1/1 [==============================] - 0s 109ms/step
Rekomendasi makanan bedasarkan kategori 'Jajanan' dengan rasa 'asin, gurih, renyah':
1. Lumpia
2. Tempura
3. Emping
4. kentang Goreng
5. Roti Bakar Keju


In [12]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Fungsi untuk menangani aksi tombol submit
def submit_action(b):
    category = category_dropdown.value
    taste = taste_input.value
    recommended_makanan = recommend_makanan(category, taste)

    # Membersihkan output sebelumnya
    with output:
        clear_output(wait=True)

        # Menampilkan hasil rekomendasi makanan
        print(f"Rekomendasi makanan bedasarkan kategori '{category}' dengan rasa '{taste}':")
        for makanan in recommended_makanan:
            print(f"- {makanan}")


# Membuat dropdown untuk kategori
category_options = ['Makanan Berat', 'Makanan Ringan', 'Jajanan', 'Minuman']
category_dropdown = widgets.Dropdown(
    options=category_options,
    value=category_options[0],
    description='Category:',
    disabled=False,
)

# Membuat kolom input untuk rasa
taste_input = widgets.Text(
    value='',
    placeholder='Enter taste',
    description='Taste:',
    disabled=False
)

# Membuat tombol submit
submit_button = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

# Membuat widget output untuk menampilkan hasil
output = widgets.Output()

# Menambahkan event handler untuk tombol submit
submit_button.on_click(submit_action)

# Menampilkan dropdown kategori, kolom input rasa, dan tombol submit
display(category_dropdown, taste_input, submit_button, output)

Dropdown(description='Category:', options=('Makanan Berat', 'Makanan Ringan', 'Jajanan', 'Minuman'), value='Ma…

Text(value='', description='Taste:', placeholder='Enter taste')

Button(description='Submit', icon='check', style=ButtonStyle(), tooltip='Click me')

Output()

#Rekomendasi restoran bedasarkan makanan yang dipilih

In [13]:
makanan_data['Features'] = makanan_data['makanan/minuman']

# Encode data teks menggunakan TF-IDF
tfidf = TfidfVectorizer()
tfidf_matrix_mamin= tfidf.fit_transform(makanan_data['Features'])

Cosine Similarity

In [14]:
# Menangani nilai NaN dalam kolom 'variasi_makanan'
restoran_data['variasi_makanan'].fillna('', inplace=True)

# matrix TF-IDF untuk data restoran
tfidf_matrix_restaurant = tfidf.transform(restoran_data['variasi_makanan'])

# Menghitung cosine similarity antara matriks TF-IDF makanan dan restoran
cosine_sim = cosine_similarity(tfidf_matrix_mamin, tfidf_matrix_restaurant)

# Fungsi untuk merekomendasikan restoran bedasarkan makanan yang dipilih
def recommend_restaurants_for_food_containing_keyword(keyword, cosine_sim, makanan_data, restoran_data):
    # Menemukan indeks makanan yang mengandung kata kunci
    food_indices = makanan_data[makanan_data['makanan/minuman'].str.contains(keyword, case=False, na=False)].index.tolist()

    # Daftar restoran yang direkomendasikan dengan skor cosine similarity
    recommended_restaurants_with_scores = []

    for food_index in food_indices:
        # Hitung dan urutkan skor cosine similarity untuk setiap makanan
        sim_scores = list(enumerate(cosine_sim[food_index]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Mendapatkan 5 restoran teratas yang paling mirip dengan skor cosine similarity untuk setiap makanan
        top_similar_restaurants = sim_scores[:5]

        for idx, score in top_similar_restaurants:
            recommended_restaurants_with_scores.append((restoran_data.iloc[idx]['nama_restoran'],
                                                        score, makanan_data.iloc[food_index]['makanan/minuman']))

    # Mengurutkan daftar akhir bedasarkan skor cosine similarity
    recommended_restaurants_with_scores.sort(key=lambda x: x[1], reverse=True)

    # Mendapatkan rekomendasi unik untuk menghindari adanya duplikasi
    unique_recommendations = []
    seen_restaurants = set()
    for rec in recommended_restaurants_with_scores:
        if rec[0] not in seen_restaurants:
            seen_restaurants.add(rec[0])
            unique_recommendations.append(rec)

    return unique_recommendations[:5]

<ipython-input-14-6d2ef7f6225e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  restoran_data['variasi_makanan'].fillna('', inplace=True)


Inputan makanan yang dipilih

In [15]:
# Memilih makanan
selected_food = 'kue adrem'

# Merekomendasikan restoran
recommended_restaurants_with_scores = recommend_restaurants_for_food_containing_keyword(selected_food, cosine_sim, makanan_data, restoran_data)

print(f"Rekomendasi restoran yang menyediakan makanan/minuman yang memiliki menu '{selected_food}' :")
for nama_restoran, score, variasi_makanan in recommended_restaurants_with_scores:
    print(f"- {nama_restoran} menyediakan {variasi_makanan}")

Rekomendasi restoran yang menyediakan makanan/minuman yang memiliki menu 'kue adrem' :
- Lupis Mbah Satinem  menyediakan Kue Adrem 
- Kipo Bu Djito ꦏꦶꦥꦺꦴꦧꦸꦣ꧀ꦗꦶꦠꦺꦴ menyediakan Kue Adrem 
- Panada & Puding RoBin menyediakan Kue Adrem 
- Kue Bandung & Pukis Ko Ayung Kotabaru menyediakan Kue Adrem 
- Lapis Legit Langgeng Sari menyediakan Kue Adrem 


In [16]:
model.save('modelrecom.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [17]:
# Simpan file .pkl

import pickle
with open('modelrecom.pkl','wb') as file:
  pickle.dump(model, file)